In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

/Users/gengsudev/Documents/DEV/AgenticAI_LangGraph-LangChain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
speech = TextLoader("../1.data-ingestion/speech.txt", autodetect_encoding=True).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
speech_chunks = text_splitter.split_documents(speech)

In [11]:
speech_chunks

[Document(metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no'),
 Document(metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied'),
 Document(metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall

In [13]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
docs = embeddings.embed_documents([chunk.page_content for chunk in speech_chunks])

In [17]:
db = FAISS.from_documents(speech_chunks, embeddings)

In [18]:
query = "How does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
print(docs[0].page_content)

ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is


In [19]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is'

In [21]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score
# lower score means more similar

[(Document(id='a605c9cd-913d-427c-b2cc-ed124e493b77', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is'),
  np.float32(0.7098052)),
 (Document(id='7b5620ca-7245-4cfc-8916-68687de06ea8', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our'),
  np.float32(0.8568388)),
 (Document(id='1b1d77af-66d6-4709-a84b-99c441c8e9b6', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no domini

In [22]:
embeddings_vector = embeddings.embed_query(query)
embeddings_vector

[-0.025556335,
 -0.0005634181,
 -0.16197641,
 -0.026059695,
 0.10034263,
 0.067196324,
 -0.042196028,
 -0.0069123073,
 0.019592393,
 -0.02863729,
 -0.011116316,
 0.042366385,
 0.036743443,
 0.066345,
 0.09461707,
 -0.04849169,
 0.024775924,
 -0.05161969,
 -0.045822777,
 0.029538449,
 -0.045873422,
 -0.014869954,
 0.0007630678,
 0.017366584,
 0.088581055,
 0.02943011,
 -0.012550471,
 0.041677736,
 -0.04451649,
 -0.021120824,
 0.051389035,
 -0.014862145,
 -0.011841105,
 0.007681556,
 -0.09053899,
 -0.069537215,
 0.023963232,
 0.046544164,
 0.020850237,
 -0.06350313,
 0.006483253,
 -0.013997891,
 0.0015952192,
 -0.07046655,
 0.060861226,
 -0.006249512,
 -0.0028105695,
 0.01515078,
 0.009400249,
 -0.029958135,
 -0.030160239,
 -0.010332611,
 0.013549239,
 -0.0011877244,
 0.053545333,
 0.012265677,
 -0.022402134,
 -0.03947883,
 0.0121757025,
 0.014617895,
 0.026440084,
 0.013485726,
 -0.027633337,
 0.050461035,
 0.057567716,
 -0.048598267,
 -0.051452663,
 0.025600666,
 0.029826552,
 -0.07423

In [24]:
docs_score = db.similarity_search_by_vector(embeddings_vector)
docs_score

[Document(id='a605c9cd-913d-427c-b2cc-ed124e493b77', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is'),
 Document(id='7b5620ca-7245-4cfc-8916-68687de06ea8', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our'),
 Document(id='1b1d77af-66d6-4709-a84b-99c441c8e9b6', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no'),
 Document(id='a384059f-ff3a-4c4d-b0e9

# I/O


In [27]:
db.save_local("faiss_ollama_speech")

In [29]:
db_loaded = FAISS.load_local(
    "faiss_ollama_speech", embeddings=embeddings, allow_dangerous_deserialization=True
)

In [30]:
db_loaded.similarity_search(query)

[Document(id='a605c9cd-913d-427c-b2cc-ed124e493b77', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is'),
 Document(id='7b5620ca-7245-4cfc-8916-68687de06ea8', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our'),
 Document(id='1b1d77af-66d6-4709-a84b-99c441c8e9b6', metadata={'source': '../1.data-ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no'),
 Document(id='a384059f-ff3a-4c4d-b0e9

## Key Differences

**`embeddings.embed_documents()`** - Manual embedding

- Returns a **list of vectors** (numerical arrays)
- Just the raw embeddings, no storage or indexing
- You need to manually handle storage, indexing, and similarity search
- Lower-level control but more work

**`FAISS.from_documents()`** - Automatic vector store creation

- Creates a **complete vector store object** with built-in indexing
- Embeds the documents AND stores them in a FAISS index
- Provides ready-to-use similarity search methods
- Handles the relationship between vectors, documents, and metadata

## Practical Example

```python
# Manual approach - just embeddings
docs = embeddings.embed_documents([chunk.page_content for chunk in speech_chunks])
# docs = [[0.1, 0.5, ...], [0.3, 0.2, ...], ...]
# Now what? You'd need to manually implement search

# Vector store approach - complete solution
vectorstore = FAISS.from_documents(speech_chunks, embeddings)
# Now you can immediately do:
results = vectorstore.similarity_search("your query", k=3)
# Returns the actual Document objects, not just vectors
```

## What FAISS.from_documents() Does Behind the Scenes

1. Extracts text from each document (`chunk.page_content`)
2. Calls `embeddings.embed_documents()` on them
3. Creates a FAISS index structure for efficient similarity search
4. Stores the original documents and metadata alongside vectors
5. Returns a searchable vectorstore object

## When to Use Each

**Use `embed_documents()` when:**

- You want to store vectors in a custom database
- You need fine-grained control over the indexing process
- You're building a custom retrieval system

**Use `FAISS.from_documents()` when:**

- You want a quick, working RAG system
- You need standard similarity search functionality
- You want LangChain integration for retrieval chains

The second approach is more convenient for most RAG applications because it gives you a ready-to-use retrieval system rather than just raw vectors.
